### ДЗ Урок 4.

Данное ДЗ является продолжением ДЗ №3, так как требуется добавление распарсенных вакансий в БД MongoDB. Поэтому сначала дублируется код предыдущего ДЗ для получения списка вакансий.

In [1]:
from bs4 import BeautifulSoup as bs
from pprint import pprint
import requests
from time import sleep

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
}

url = 'https://voronezh.hh.ru/vacancies/razrabotchik'

In [3]:
def parse(url_page, headers, result=[], index_page=1):
    
    response = requests.get(url_page, headers=headers)
    
    if response.status_code != 200:
      print('Parsing ended')
      return result
    else:
      print(f'Page{index_page} {response.url}')

    dom = bs(response.content, 'html.parser')
    vacancies = dom.find_all('div', {'class': 'vacancy-serp-item__layout'})
    for vacancy in vacancies:
      result.append(parse_vacancy(vacancy))

    link_next_page = dom.find('a', {'data-qa': 'pager-next'})
    if link_next_page:
      link_next_page =  'https://voronezh.hh.ru' + link_next_page['href']
    else:
      print('Parsing ended')
      return result

    result = parse(link_next_page, headers, result, index_page+1)   
    return result
     
def parse_vacancy(dom_vacancy):
  sleep(0.2)
  vacancy_name = dom_vacancy.find('a').text

  vacancy_salary = dom_vacancy.find('span', {'class', 'bloko-header-section-3'})
  if vacancy_salary:
    vacancy_salary = vacancy_salary.text
    min_salary, max_salary, currency_salary = clean_salary(vacancy_salary)
  else:
    min_salary, max_salary, currency_salary = None, None, None

  vacancy_link = dom_vacancy.find('a')['href']
  
  return {
      'name': vacancy_name,
      'salary': vacancy_salary,
      'min_salary': min_salary,
      'max_salary': max_salary,
      'currency_salary': currency_salary,
      'link': vacancy_link,
      'source': 'hh.ru',
  }

def clean_salary(vacancy_salary_text, min_salary=None, max_salary=None, currency_salary=None):
  list_salary = vacancy_salary_text.replace('\u202f', '').split()
  for i in range(len(list_salary) - 1):
    if list_salary[i] == 'от':
      min_salary = int(list_salary[i + 1])
    elif list_salary[i] == 'до':
      max_salary = int(list_salary[i + 1])
    elif list_salary[i] == '–':
      min_salary = int(list_salary[i - 1])
      max_salary = int(list_salary[i + 1])
  currency_salary = list_salary[-1]

  return min_salary, max_salary, currency_salary

In [4]:
result = parse(url, headers)

Page1 https://voronezh.hh.ru/vacancies/razrabotchik
Page2 https://voronezh.hh.ru/vacancies/razrabotchik?page=1&hhtmFrom=vacancy_search_catalog
Page3 https://voronezh.hh.ru/vacancies/razrabotchik?page=2&hhtmFrom=vacancy_search_catalog
Page4 https://voronezh.hh.ru/vacancies/razrabotchik?page=3&hhtmFrom=vacancy_search_catalog
Page5 https://voronezh.hh.ru/vacancies/razrabotchik?page=4&hhtmFrom=vacancy_search_catalog
Page6 https://voronezh.hh.ru/vacancies/razrabotchik?page=5&hhtmFrom=vacancy_search_catalog
Page7 https://voronezh.hh.ru/vacancies/razrabotchik?page=6&hhtmFrom=vacancy_search_catalog
Page8 https://voronezh.hh.ru/vacancies/razrabotchik?page=7&hhtmFrom=vacancy_search_catalog
Page9 https://voronezh.hh.ru/vacancies/razrabotchik?page=8&hhtmFrom=vacancy_search_catalog
Page10 https://voronezh.hh.ru/vacancies/razrabotchik?page=9&hhtmFrom=vacancy_search_catalog
Parsing ended


In [5]:
result

[{'name': 'Backend-разработчик / Python developer',
  'salary': '100\u202f000 – 200\u202f000 руб.',
  'min_salary': 100000,
  'max_salary': 200000,
  'currency_salary': 'руб.',
  'link': 'https://voronezh.hh.ru/vacancy/74109649?query=%D1%80%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA',
  'source': 'hh.ru'},
 {'name': 'Разработчик Python',
  'salary': None,
  'min_salary': None,
  'max_salary': None,
  'currency_salary': None,
  'link': 'https://voronezh.hh.ru/vacancy/73997850?query=%D1%80%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA',
  'source': 'hh.ru'},
 {'name': 'Full stack разработчик',
  'salary': 'от 80\u202f000 руб.',
  'min_salary': 80000,
  'max_salary': None,
  'currency_salary': 'руб.',
  'link': 'https://voronezh.hh.ru/vacancy/73581131?query=%D1%80%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA',
  'source': 'hh.ru'},
 {'name': 'JavaScript разработчик',
  'salary': None,
  'min_salary': None,
  'max_salary': None,
  'currency_sal

#### Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.

Сервер MongoDb развернул на windows. Настройки по умолчанию. pymongo установил через менеджер Anaconda.

In [6]:
from pymongo import MongoClient

In [7]:
client = MongoClient()

In [8]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

Создаем БД

In [15]:
db = client.jobs

Создаем коллекцию

In [16]:
db_vacancies = db.vacancies

In [17]:
def insert_new_vacancies(db_vacancies, vacancies):
  for vacancy in vacancies:
    if not len(list(db_vacancies.find({'link': vacancy['link']}))):
      db_vacancies.insert_one(vacancy)

In [19]:
insert_new_vacancies(db_vacancies, result)

In [21]:
for vacancy in db_vacancies.find():
    pprint(vacancy)

{'_id': ObjectId('63ac9c265c09039dffa3f5d2'),
 'currency_salary': 'руб.',
 'link': 'https://voronezh.hh.ru/vacancy/74109649?query=%D1%80%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA',
 'max_salary': 200000,
 'min_salary': 100000,
 'name': 'Backend-разработчик / Python developer',
 'salary': '100\u202f000 – 200\u202f000 руб.',
 'source': 'hh.ru'}
{'_id': ObjectId('63ac9c265c09039dffa3f5d3'),
 'currency_salary': None,
 'link': 'https://voronezh.hh.ru/vacancy/73997850?query=%D1%80%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA',
 'max_salary': None,
 'min_salary': None,
 'name': 'Разработчик Python',
 'salary': None,
 'source': 'hh.ru'}
{'_id': ObjectId('63ac9c265c09039dffa3f5d4'),
 'currency_salary': 'руб.',
 'link': 'https://voronezh.hh.ru/vacancy/73581131?query=%D1%80%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA',
 'max_salary': None,
 'min_salary': 80000,
 'name': 'Full stack разработчик',
 'salary': 'от 80\u202f000 руб.',
 'source': 'hh.ru'}

{'_id': ObjectId('63ac9c275c09039dffa3f696'),
 'currency_salary': 'руб.',
 'link': 'https://voronezh.hh.ru/vacancy/72235440?query=%D1%80%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA',
 'max_salary': None,
 'min_salary': 30000,
 'name': 'Специалист по внедрению и сопровождению ПО',
 'salary': 'от 30\u202f000 руб.',
 'source': 'hh.ru'}
{'_id': ObjectId('63ac9c275c09039dffa3f697'),
 'currency_salary': 'руб.',
 'link': 'https://voronezh.hh.ru/vacancy/72168394?query=%D1%80%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA',
 'max_salary': None,
 'min_salary': 130000,
 'name': 'Программист 1 С (ЗУП; КОРП)',
 'salary': 'от 130\u202f000 руб.',
 'source': 'hh.ru'}
{'_id': ObjectId('63ac9c275c09039dffa3f698'),
 'currency_salary': 'руб.',
 'link': 'https://voronezh.hh.ru/vacancy/72657273?query=%D1%80%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA',
 'max_salary': 183908,
 'min_salary': 160919,
 'name': 'Консультант 1С:ЗУП (удаленно)',
 'salary': '160\u202f919

 'name': 'System analyst',
 'salary': None,
 'source': 'hh.ru'}
{'_id': ObjectId('63ac9c275c09039dffa3f764'),
 'currency_salary': 'руб.',
 'link': 'https://voronezh.hh.ru/vacancy/73503011?query=%D1%80%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA',
 'max_salary': None,
 'min_salary': 130000,
 'name': 'Аналитик',
 'salary': 'от 130\u202f000 руб.',
 'source': 'hh.ru'}
{'_id': ObjectId('63ac9c275c09039dffa3f765'),
 'currency_salary': None,
 'link': 'https://voronezh.hh.ru/vacancy/73530375?query=%D1%80%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA',
 'max_salary': None,
 'min_salary': None,
 'name': 'DevOps specialist (Девопс)',
 'salary': None,
 'source': 'hh.ru'}
{'_id': ObjectId('63ac9c275c09039dffa3f766'),
 'currency_salary': 'руб.',
 'link': 'https://voronezh.hh.ru/vacancy/73493055?query=%D1%80%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA',
 'max_salary': 39000,
 'min_salary': 15000,
 'name': 'Специалист службы поддержки с техническими знания

#### Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы.

In [25]:
def find_vacancies(db_vacancies, min_salary):
    for vacancy in db_vacancies.find({"min_salary": {"$gt":min_salary}}):
        pprint(vacancy)    

In [26]:
find_vacancies(db_vacancies, 100000)

{'_id': ObjectId('63ac9c265c09039dffa3f5d6'),
 'currency_salary': 'руб.',
 'link': 'https://voronezh.hh.ru/vacancy/73351087?query=%D1%80%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA',
 'max_salary': 150000,
 'min_salary': 150000,
 'name': 'Разработчик PostgreSQL (удаленно)',
 'salary': '150\u202f000 – 150\u202f000 руб.',
 'source': 'hh.ru'}
{'_id': ObjectId('63ac9c265c09039dffa3f5dc'),
 'currency_salary': 'руб.',
 'link': 'https://voronezh.hh.ru/vacancy/73315066?query=%D1%80%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA',
 'max_salary': 231000,
 'min_salary': 138000,
 'name': 'Руководитель отдела Web разработки',
 'salary': '138\u202f000 – 231\u202f000 руб.',
 'source': 'hh.ru'}
{'_id': ObjectId('63ac9c265c09039dffa3f5e2'),
 'currency_salary': 'руб.',
 'link': 'https://voronezh.hh.ru/vacancy/73982417?query=%D1%80%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA',
 'max_salary': 190000,
 'min_salary': 132000,
 'name': 'Программист PHP/ Разработчик